In [1]:
import pandas as pd
import cufflinks as cf
import numpy as np

cf.go_offline()

In [273]:
sima=pd.read_pickle('Repe.pkl')

In [4]:
reno=pd.read_pickle('Sin_repe.pkl')

In [8]:
ca=pd.read_pickle('Circulo_Azul.pkl')

In [11]:
df=pd.read_csv('Motivos_R09_trans.csv', sep='\t')

In [16]:
df=df[df['linea']!=5500000000]

In [17]:
df=df[df['linea'].map(lambda x: len(str(x))==10)]

In [22]:
sima['motivo'].value_counts().index

Index(['T RENOVACION (CIRCULO AZUL)', 'T COTIZACION DE PUNTOS DE CIRCULO AZUL',
       'T POR CIRCULO AZUL', 'C CAMBIO DE EQUIPO (RENOVACION)'],
      dtype='object')

In [26]:
df1=df[df['motivo'].isin(sima['motivo'].value_counts().index)]

In [29]:
df1['fecha']=pd.to_datetime(df1['fecha'])

<ipython-input-29-c6d40a80da76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['fecha']=pd.to_datetime(df1['fecha'])


In [30]:
df1.groupby(df1['fecha'].dt.to_period('M'))['linea'].count()

fecha
2021-08    57649
2021-09    54281
Freq: M, Name: linea, dtype: int64

In [31]:
sima['fecha'].value_counts()

2021-09    58507
2021-08    56427
Name: fecha, dtype: int64

In [224]:
histo1=pd.DataFrame([reno[(reno['renovacion']==0)&(reno['fecha']=='2021-08')]['meses_restantes'],reno[(reno['renovacion']==0)&(reno['fecha']=='2021-09')]['meses_restantes']], index=['Agosto','Septiembre']).T

In [225]:
histo1.iplot('hist', title='Adendum CAC', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas',asUrl=True)

'Adendum CAC.html'

In [62]:
histo1.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9]).to_excel('Adendum.xlsx')

In [83]:
telea=pd.read_csv('Telefonico_adm_ag.csv',sep='\t')
teles=pd.read_csv('Telefonico_adm_sep.csv',sep='\t')

In [90]:
telea=telea[telea['feadendum']!='(null)']
teles=teles[teles['feadendum']!='(null)']

In [94]:
telea['feadendum']=pd.to_datetime(telea['feadendum'])
telea['fefinadendum']=pd.to_datetime(telea['fefinadendum'])
teles['feadendum']=pd.to_datetime(teles['feadendum'])
teles['fefinadendum']=pd.to_datetime(teles['fefinadendum'])

In [97]:
telea['mes']=pd.to_datetime('2021-08-01')
teles['mes']=pd.to_datetime('2021-09-01')
tele=pd.concat([telea,teles])

In [101]:
tele['renovacion']=(tele['feadendum'].dt.to_period('M')>='2021-08').astype(int)

In [106]:
tele['meses_restantes']=((tele['mes']-tele['fefinadendum'])/np.timedelta64(1, 'M')).round()

In [194]:
tele

,linea,feadendum,fefinadendum,mes,renovacion,meses_restantes
0,7731093178,2021-08-20,2023-08-20,2021-08-01,1,-25.0
1,5529151687,2021-03-25,2023-03-25,2021-08-01,0,-20.0
2,5544484508,2019-08-18,2021-08-18,2021-08-01,0,-1.0
3,5562322038,2021-08-16,2023-08-16,2021-08-01,1,-24.0
4,5579251197,2020-12-23,2022-12-23,2021-08-01,0,-17.0
...,...,...,...,...,...,...
6431,7712029342,2019-11-20,2021-11-20,2021-09-01,0,-3.0
6432,5551861559,2021-09-08,2023-09-08,2021-09-01,1,-24.0
6433,7121428747,2019-12-16,2021-12-16,2021-09-01,0,-3.0
6434,5540641981,2019-03-30,2021-03-30,2021-09-01,0,5.0


In [112]:
histo2=pd.DataFrame([tele[(tele['renovacion']==0)&(tele['mes']=='2021-08')]['meses_restantes'],tele[(tele['renovacion']==0)&(tele['mes']=='2021-09')]['meses_restantes']], index=['Agosto','Septiembre']).T

In [127]:
sima[sima['fecha']=='2021-08']['linea'].drop_duplicates().to_csv('lineas_agosto.csv',index=False)

In [128]:
sima[sima['fecha']=='2021-09']['linea'].drop_duplicates().to_csv('lineas_septiembre.csv',index=False)

In [275]:
tele[tele['renovacion']==0]['meses_restantes'].iplot('hist', title='Adendum CAT', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas',bins=(-28,40,1))

In [239]:
tele['puede_renovar']=(tele['meses_restantes']>=-3).astype(int)

In [242]:
aux=tele[tele['renovacion']==0]

In [246]:
aux.groupby([aux['mes'].dt.to_period('M'),'puede_renovar'])['linea'].count().unstack().to_excel('No_Reno_Cat.xlsx')

In [255]:
tele.groupby([tele['mes'].dt.to_period('M'),'renovacion'])['linea'].count().unstack().to_excel('Tasa_renovacion_cat.xlsx')

In [113]:
histo2.iplot('hist', title='Adendum CAT', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas',asUrl=True)

'Adendum CAT.html'

In [114]:
histo2.iplot('hist', title='Adendum CAT', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas')

In [115]:
histo1.iplot('hist', title='Adendum CAT', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas')

In [159]:
l=[]
for i in ['SIMAEC_adm_ag.csv','SIMAEC_adm_ag1.csv','SIMAEC_adm_sep.csv','SIMAEC_adm_sep1.csv']:
    l.append(pd.read_csv(i,sep='\t')) 

In [160]:
aug=pd.concat(l[0:2])

In [161]:
aug['fecha']=pd.to_datetime('2021-08-01')

In [162]:
sep=pd.concat(l[2:])

In [163]:
sep['fecha']=pd.to_datetime('2021-09-01')

In [164]:
lin=pd.concat([aug,sep])

In [165]:
lin['fecha']=lin['fecha'].dt.to_period('M')

In [186]:
lin=lin[lin['feadendum']!='(null)']

In [187]:
lin['fefinadendum']=pd.to_datetime(lin['fefinadendum'])
lin['feadendum']=pd.to_datetime(lin['feadendum'])

In [188]:
lin

,linea,feadendum,fefinadendum,fecha
0,7228228535,2021-08-13,2023-08-13,2021-08
1,5516495142,2020-07-07,2021-07-07,2021-08
2,7751180193,2020-09-24,2021-09-24,2021-08
3,5526652209,2021-08-14,2023-08-14,2021-08
4,5554063105,2021-08-17,2022-08-17,2021-08
...,...,...,...,...
28637,7771613335,2019-10-02,2021-10-02,2021-09
28638,5560090009,2019-10-25,2021-10-25,2021-09
28639,5554076376,2021-09-04,2023-09-04,2021-09
28640,5514748964,2019-11-07,2021-11-07,2021-09


In [170]:
reno.drop(['feadendum','meses_adm','fefinadendum','plazo_plan','meses_restantes','puede_renovar'],axis=1,inplace=True)

In [179]:
reno['fecha']=pd.to_datetime(reno['fecha']).dt.to_period('M')

In [190]:
reno=reno.merge(lin, how='left',on=['linea','fecha'])

0        2021-08
1        2021-08
2        2021-08
3        2021-08
4        2021-08
          ...   
96230    2021-09
96231    2021-09
96232    2021-09
96233    2021-09
96234    2021-09
Name: fecha, Length: 96235, dtype: object

In [210]:
reno['meses_restantes']=((pd.to_datetime(reno['fecha'].astype(str))-reno['fefinadendum'])/np.timedelta64(1, 'M')).round()

In [213]:
(reno['feadendum']>='2021-08-01').sum()

49964

In [268]:
reno[(reno['renovacion']==0)]['meses_restantes'].iplot('hist', title='Adendum CAC', xTitle='Meses Fin de Adendum',yTitle='Cantidad de Líneas',bins=(-28,40,1),asUrl=True)

'Adendum CAC.html'

In [116]:
histo2.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9]).to_excel('Adendum_CAT.xlsx')

In [80]:
reno=reno[reno['feadendum'].notna()]

In [69]:
reno['renovacion_circa']=reno['linea'].isin(ca['LINEA'].unique()).astype(int)

In [77]:
((reno['renovacion_circa']==1)&(reno['feadendum'].isna())).sum()

158

In [79]:
reno['feadendum'].isna().sum()

7306

In [60]:
ca[ca[' IDREGION']==9][' FECHAOPERACION'].dt.to_period('M').value_counts()

2021-08    68911
2021-09    60054
Freq: M, Name:  FECHAOPERACION, dtype: int64

In [121]:
reno[reno['renovacion']==0].groupby(['fecha','puede_renovar'])['linea'].count().unstack().to_excel('No_Reno.xlsx')

In [226]:
reno

,linea,region,fecha,cac,motivo,NPS,renovacion,renovacion_cac,renovacion_otro,Medio,renovacion_circa,feadendum,fefinadendum,meses_restantes
0,5518307165,9,2021-08,PLAZA LORETO,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,0,0,0,OTRO,0,2019-11-18,2021-11-18,-4.0
1,5545402887,9,2021-08,CAMARONES,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,0,0,0,OTRO,0,2019-05-23,2021-05-23,2.0
2,5518343872,9,2021-08,TLALNEPANTLA,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,0,0,0,OTRO,0,2019-10-16,2021-10-16,-2.0
3,5541353325,9,2021-08,PARQUE DELTA,T RENOVACION (CIRCULO AZUL),NaN,0,0,0,OTRO,0,2019-04-16,2021-04-16,4.0
4,5525044545,9,2021-08,LAS AMERICAS,T RENOVACION (CIRCULO AZUL),NaN,1,1,0,CAC,1,2021-08-13,2023-08-13,-24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96230,5574317433,9,2021-09,GALERIAS COAPA,T RENOVACION (CIRCULO AZUL),NaN,0,0,0,OTRO,0,2020-11-18,2021-11-18,-3.0
96231,7121485213,9,2021-09,ATLACOMULCO,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,0,0,0,OTRO,0,2019-09-22,2021-09-22,-1.0
96232,7712213013,9,2021-09,GALERIAS PACHUCA,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,0,0,0,OTRO,0,2019-02-17,2021-02-17,6.0
96233,5554067181,9,2021-09,MASARYK,T RENOVACION (CIRCULO AZUL),NaN,1,1,0,CAC,1,2021-09-05,2023-09-05,-24.0


In [117]:
tele.to_csv('Telefonico_ag_sept.csv',index=False)

In [229]:
reno=reno[reno['feadendum'].notna()]

In [233]:
reno['puede_renovar']=(reno['meses_restantes']<-3).astype(int)

In [235]:
reno['renovacion']=reno['feadendum']>='2021-08-01'

In [236]:
reno

,linea,region,fecha,cac,motivo,NPS,renovacion,renovacion_cac,renovacion_otro,Medio,renovacion_circa,feadendum,fefinadendum,meses_restantes,puede_renovar
0,5518307165,9,2021-08,PLAZA LORETO,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,False,0,0,OTRO,0,2019-11-18,2021-11-18,-4.0,1
1,5545402887,9,2021-08,CAMARONES,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,False,0,0,OTRO,0,2019-05-23,2021-05-23,2.0,0
2,5518343872,9,2021-08,TLALNEPANTLA,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,False,0,0,OTRO,0,2019-10-16,2021-10-16,-2.0,0
3,5541353325,9,2021-08,PARQUE DELTA,T RENOVACION (CIRCULO AZUL),NaN,False,0,0,OTRO,0,2019-04-16,2021-04-16,4.0,0
4,5525044545,9,2021-08,LAS AMERICAS,T RENOVACION (CIRCULO AZUL),NaN,True,1,0,CAC,1,2021-08-13,2023-08-13,-24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96230,5574317433,9,2021-09,GALERIAS COAPA,T RENOVACION (CIRCULO AZUL),NaN,False,0,0,OTRO,0,2020-11-18,2021-11-18,-3.0,0
96231,7121485213,9,2021-09,ATLACOMULCO,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,False,0,0,OTRO,0,2019-09-22,2021-09-22,-1.0,0
96232,7712213013,9,2021-09,GALERIAS PACHUCA,T COTIZACION DE PUNTOS DE CIRCULO AZUL,NaN,False,0,0,OTRO,0,2019-02-17,2021-02-17,6.0,0
96233,5554067181,9,2021-09,MASARYK,T RENOVACION (CIRCULO AZUL),NaN,True,1,0,CAC,1,2021-09-05,2023-09-05,-24.0,1


In [237]:
reno.to_csv('Renovaciones_final_ahora_si.csv',index=False)

## Renovaciones 2021

In [321]:
df=pd.read_csv('SIMAEC_adm_2021.csv',sep='\t')

In [322]:
df['fecha']=pd.to_datetime(df['fecha'])
df['feadendum_insitu']=pd.to_datetime(df['feadendum_insitu'])
df['feadendum_actual']=pd.to_datetime(df['feadendum_actual'])

In [323]:
df=df[df['feadendum_insitu']!='1990-01-01']

In [324]:
df=df[df['feadendum_actual']<'2021-10-31']

In [325]:
df['renovacion_insitu']=(((df['feadendum_actual']-df['fecha'])/np.timedelta64(1, 'D')).between(-30,30)).astype(int)

In [516]:
aux=df.groupby([df['fecha'].dt.to_period('M'),'renovacion_insitu'])['linea'].count().unstack()

In [517]:
aux.columns=['No','Sí']

In [518]:
aux['Tasa de Renovación']=(aux['Sí']*100/(aux.sum(axis=1))).round(2)

In [519]:
aux

,No,Sí,Tasa de Renovación
fecha,,,
2021-01,15630,35759,69.58
2021-02,17557,38714,68.80
2021-03,21690,43604,66.78
2021-04,18245,35931,66.32
2021-05,21080,39737,65.34
2021-06,20278,34868,63.23
2021-07,21801,35091,61.68
2021-08,23327,32230,58.01
2021-09,30938,26674,46.30


## Tasa de Renovación 

In [520]:
aux['Tasa de Renovación'].iplot(title='Tasa de Renovación Insitu', xTitle='Mes',yTitle='Clientes',colorscale='ggplot', asUrl=True)

'Tasa de Renovación Insitu.html'

## Renovaciones

In [521]:
aux[['No','Sí']].iplot(title='Renovaciones Insitu', xTitle='Mes',yTitle='Clientes',colorscale='ggplot',asUrl=True)

'Renovaciones Insitu.html'

In [522]:
df['meses_despues']=((df['feadendum_actual']-df['fecha'])/np.timedelta64(1, 'M')).round()

In [523]:
df['finadm_insitu']=(df['feadendum_insitu']+df['periodo_adm_insitu'].values.astype("timedelta64[M]"))

In [524]:
df['meses_restantes']=((df['fecha']-df['finadm_insitu'])/np.timedelta64(1, 'M')).round()

In [525]:
aux=df.groupby([df['fecha'].dt.to_period('M'),'renovacion'])['linea'].count().unstack()

In [526]:
aux.columns=['No','Sí']

In [527]:
aux['Tasa de Renovación']=(aux['Sí']*100/(aux.sum(axis=1))).round(2)

In [528]:
aux

,No,Sí,Tasa de Renovación
fecha,,,
2021-01,9089,42300,82.31
2021-02,10774,45497,80.85
2021-03,14016,51278,78.53
2021-04,12247,41929,77.39
2021-05,15172,45645,75.05
2021-06,15950,39196,71.08
2021-07,18385,38507,67.68
2021-08,21366,34191,61.54
2021-09,30372,27240,47.28


## Tasa de Renovación 

In [529]:
aux['Tasa de Renovación'].iplot(title='Tasa de Renovación Total', xTitle='Mes',yTitle='Clientes',colorscale='ggplot', asUrl=True)

'Tasa de Renovación Total.html'

## Renovaciones

In [530]:
aux[['No','Sí']].iplot(title='Renovaciones Totales', xTitle='Mes',yTitle='Clientes',colorscale='ggplot',asUrl=True)

'Renovaciones Totales.html'

# Política de 90 días

In [531]:
df[(df['renovacion_insitu']==0)&(df['meses_restantes']<40)]['meses_restantes'].iplot('hist',title='Política de 90 días', xTitle='Meses por Fin de Adendum',yTitle='Clientes',colorscale='ggplot',asUrl=True)

'Política de 90 días.html'

In [552]:
df[(df['renovacion_insitu']==0)&(df['meses_restantes']<40)]['meses_restantes']

13       -10.0
14        -2.0
15        -3.0
16        -9.0
17         7.0
          ... 
570190     1.0
570241    -1.0
570242    -3.0
570243     1.0
570244    -3.0
Name: meses_restantes, Length: 226485, dtype: float64

In [553]:
df[(df['renovacion_insitu']==0)]['meses_restantes'].describe()

count    229391.000000
mean          1.495364
std          11.163551
min         -35.000000
25%          -3.000000
50%           0.000000
75%           4.000000
max         165.000000
Name: meses_restantes, dtype: float64

In [532]:
df['Afectados por Política de 90 Días']=(df['meses_restantes']<-3).astype(int)

In [533]:
aux=df[df['renovacion_insitu']==0]

In [534]:
aux=aux.groupby([aux['fecha'].dt.to_period('M'),'Afectados por Política de 90 Días'])['linea'].count().unstack()

In [535]:
aux.columns=['No','Sí']

In [536]:
aux['Tasa de Afectados']=(aux['Sí']*100/(aux.sum(axis=1))).round(2)

In [537]:
aux

,No,Sí,Tasa de Afectados
fecha,,,
2021-01,10537,5093,32.58
2021-02,11934,5623,32.03
2021-03,14671,7019,32.36
2021-04,13171,5074,27.81
2021-05,16412,4668,22.14
2021-06,16259,4019,19.82
2021-07,17426,4375,20.07
2021-08,19147,4180,17.92
2021-09,26772,4166,13.47


## Tasa de Afectados

In [538]:
aux['Tasa de Afectados'].iplot(title='Tasa de Afectados', xTitle='Mes',yTitle='Clientes',colorscale='ggplot',asUrl=True)

'Tasa de Afectados.html'

## Afectados

In [539]:
aux[['No','Sí']].iplot(title='Afectados', xTitle='Mes',yTitle='Clientes',colorscale='ggplot',asUrl=True)

'Afectados.html'

## Renovaron Después de N Meses

### Afectados

In [540]:
df[(df['renovacion_insitu']==0)&(df['meses_despues']>=0)&(df['Afectados por Política de 90 Días']==1)]['meses_despues'].iplot('hist',title='Renovación Posterior de Afectados', xTitle='Meses Después',yTitle='Renovaciones',colorscale='ggplot',asUrl=True)

'Renovación Posterior de Afectados.html'

### No Afectados

In [541]:
df[(df['renovacion_insitu']==0)&(df['meses_despues']>=0)&(df['Afectados por Política de 90 Días']==0)]['meses_despues'].iplot('hist',title='Renovación Posterior de No Afectados', xTitle='Meses Después',yTitle='Renovaciones',colorscale='ggplot',asUrl=True)

'Renovación Posterior de No Afectados.html'

In [344]:
df['renovacion']=((df['meses_despues']>0)|(df['renovacion_insitu']==1)).astype(int)

In [345]:
len(df[df['renovacion']==0]['linea'].unique())

150527

In [346]:
qx=pd.read_csv('Exportación de tablero-08-57-pm-2021-10-28.csv',usecols=['Survey Metadata - Start Date (-05:00 GMT)','Embedded Data - Telefono','Basado en su experiencia, ¿Qué tan probable es que recomiende el Centro de Atención a Clientes Telcel  a un amigo o familiar?','Q7 razones no resolucion'])

In [347]:
qx.columns=['fecha','NPS','linea','no_sol']

In [348]:
qx['fecha']=pd.to_datetime(pd.to_datetime(qx['fecha']).dt.date)

In [349]:
df=df.merge(qx,how='left',on=['linea','fecha'])

In [542]:
aux=df.groupby([df['fecha'].dt.to_period('M'),'renovacion_insitu','NPS'])['linea'].count().unstack()

In [543]:
def nps(df):
    df['NPS']=(df['Promotor']-df['Detractor'])*100/df.sum(axis=1)
    return df

In [544]:
aux=nps(aux)['NPS'].unstack()

In [545]:
aux.columns=['No','Sí']

## NPS Renovación Insitu

In [546]:
aux.iplot(title='NPS Renovación Insitu', xTitle='Mes',yTitle='NPS',colorscale='ggplot',asUrl=True)

'NPS Renovación Insitu.html'

# Razones de no solución

Algunos ejemplos aleatorios de que los datos no mienten...

In [393]:
aux.loc[418089]

linea                                                                       5536546995
region                                                                               9
fecha                                                              2021-06-15 00:00:00
cac                                                                          CUICUILCO
motivo                                          T COTIZACION DE PUNTOS DE CIRCULO AZUL
feadendum_actual                                                   2021-08-30 00:00:00
periodo_adm_actual                                                                  24
feadendum_insitu                                                   2019-10-21 00:00:00
periodo_adm_insitu                                                                  24
renovacion_insitu                                                                    0
meses_despues                                                                      2.0
finadm_insitu                              

In [372]:
df.loc[67122]

linea                                                                       5561140416
region                                                                               9
fecha                                                              2021-07-04 00:00:00
cac                                                                     OASIS COYOACAN
motivo                                          T COTIZACION DE PUNTOS DE CIRCULO AZUL
feadendum_actual                                                   2021-10-04 00:00:00
periodo_adm_actual                                                                  24
feadendum_insitu                                                   2019-08-10 00:00:00
periodo_adm_insitu                                                                  24
renovacion_insitu                                                                    0
meses_despues                                                                      3.0
finadm_insitu                              

In [396]:
aux[['linea','fecha','no_sol']].to_pickle('No_solucion.pkl')

In [401]:
voz=pd.read_pickle('Temas_No_sol.pkl')

In [402]:
voz

,linea,fecha,no_sol,Tema
79,7225557236,2021-03-23,No tenían el equipo en existencia,Falta de Equipo/Equipo Forzoso
100,5521287428,2021-10-14,"Quiero renovar mi plan, pero Requiero el equip...",Falta de Equipo/Equipo Forzoso
540,5540571174,2021-06-06,Por qué al parecer el sistema no le permitía a...,Fallas en el Sistema
593,5520174180,2021-09-29,"Atencion del asesor, nunca estuvo dispuesto a ...",Asesor Mal Informado/Burocracia
696,7773040403,2021-10-03,Falta de claridad y de equipos\n,Falta de Equipo/Equipo Forzoso
...,...,...,...,...
569638,5528257566,2021-09-13,Solo me comentaron que ya no podría adquirir u...,Falta de Equipo/Equipo Forzoso
569982,5537079349,2021-01-07,Fui hacer la cancelación de mi plan por que ot...,Asesor Mal Informado/Burocracia
569987,5531065286,2021-09-20,No hubo señal de internet y no recibí atención...,Mala Atención/Mal Servicio
570031,5531151067,2021-02-13,"El Telcel up es un engaño, roban el dinero y n...",Actitud


In [404]:
df['Temas']=voz['Tema']

In [ ]:
df['renovacion_insitu']

In [560]:
aux=df.groupby([df['fecha'].dt.to_period('M'),'renovacion_insitu','Temas'])['linea'].count()

In [573]:
aux.unstack().to_excel('Temas por Mes.xlsx')

In [568]:
aux.loc[:,0].unstack().iplot(title='Temas por Mes (No Renovación)', xTitle='Mes',yTitle='Cantidad de Líneas',colorscale='ggplot',asUrl=True)

'Temas por Mes (No Renovación).html'

In [569]:
aux.loc[:,1].unstack().iplot(title='Temas por Mes (Sí Renovación)', xTitle='Mes',yTitle='Cantidad de Líneas',colorscale='ggplot',asUrl=True)

'Temas por Mes (Sí Renovación).html'

In [548]:
aux.iplot(title='Temas por Mes', xTitle='Mes',yTitle='Cantidad de Líneas',colorscale='ggplot',asUrl=True)

'Temas por Mes.html'

In [424]:
help(pd.DataFrame.iplot)

Help on function _iplot in module cufflinks.plotlytools:

_iplot(self, kind='scatter', data=None, layout=None, filename='', sharing=None, title='', xTitle='', yTitle='', zTitle='', theme=None, colors=None, colorscale=None, fill=False, width=None, dash='solid', mode='', interpolation='linear', symbol='circle', size=12, barmode='', sortbars=False, bargap=None, bargroupgap=None, bins=None, histnorm='', histfunc='count', orientation='v', boxpoints=False, annotations=None, keys=False, bestfit=False, bestfit_colors=None, mean=False, mean_colors=None, categories='', x='', y='', z='', text='', gridcolor=None, zerolinecolor=None, margin=None, labels=None, values=None, secondary_y='', secondary_y_title='', subplots=False, shape=None, error_x=None, error_y=None, error_type='data', locations=None, lon=None, lat=None, asFrame=False, asDates=False, asFigure=False, asImage=False, dimensions=None, asPlot=False, asUrl=False, online=None, **kwargs)
           Returns a plotly chart either as inline char

In [426]:
cf.colors.scales()

In [554]:
df.to_csv('SIMAEC_Reno_2021_11.csv',index=False)

In [ ]:
df